# Multi-Scale Ultra High Resolution (MUR) Sea Surface Temperature (SST)


### Description

A global, gap-free, gridded, daily 1 km Sea Surface Temperature (SST) dataset created by merging multiple Level-2 satellite SST datasets. Those input datasets include the NASA Advanced Microwave Scanning Radiometer-EOS (AMSR-E), the JAXA Advanced Microwave Scanning Radiometer 2 (AMSR-2) on GCOM-W1, the Moderate Resolution Imaging Spectroradiometers (MODIS) on the NASA Aqua and Terra platforms, the US Navy microwave WindSat radiometer, the Advanced Very High Resolution Radiometer (AVHRR) on several NOAA satellites, and in situ SST observations from the NOAA iQuam project. Data are available from 2002 to present in Zarr format. The original source of the MUR data is the NASA JPL Physical Oceanography DAAC.

ref: https://registry.opendata.aws/mur/

In [2]:
import httpx
import json

endpoint = "https://t4z3zffpeh.execute-api.us-east-1.amazonaws.com/md"

#### Variables

Before we can `visualize` the dataset we HAVE TO get the list of available variables within the NetCDF file. For this we can use the `/variables` endpoint.

In [6]:
resp = httpx.get(f"{endpoint}/variables", params={"url": "s3://mur-sst/zarr-v1"}, timeout=20)
print(resp.json())

['analysed_sst', 'analysis_error', 'mask', 'sea_ice_fraction']


### Dataset Info

In [17]:
resp = httpx.get(
    f"{endpoint}/info",
    params={
        "url": "s3://mur-sst/zarr-v1",
        "variable": "analysed_sst",
        "decode_times": True,
        "show_times": True,
    },
    timeout=20,
)
print(json.dumps(resp.json(), indent=4))

{
    "bounds": [
        -179.99500549324037,
        -89.99499786365084,
        180.0050000000763,
        89.99499786365084
    ],
    "crs": "http://www.opengis.net/def/crs/EPSG/0/4326",
    "band_metadata": [
        [
            "b1",
            {}
        ]
    ],
    "band_descriptions": [
        [
            "b1",
            "value"
        ]
    ],
    "dtype": "float64",
    "nodata_type": "Nodata",
    "name": "analysed_sst",
    "count": 1,
    "width": 36000,
    "height": 17999,
    "attrs": {
        "comment": "\"Final\" version using Multi-Resolution Variational Analysis (MRVA) method for interpolation",
        "long_name": "analysed sea surface temperature",
        "standard_name": "sea_surface_foundation_temperature",
        "units": "kelvin",
        "valid_max": 32767,
        "valid_min": -32767
    }
}


### Map

In [25]:
from folium import Map, TileLayer

r = httpx.get(
    f"{endpoint}/WebMercatorQuad/tilejson.json",
    params = {
        "url": "s3://mur-sst/zarr-v1",
        "variable": "analysed_sst",
        "rescale": "250,350",  # We know the data is in kelvin
        "colormap_name": "viridis",
    },
    timeout=20,
).json()
print(r)

bounds = r["bounds"]

# The dataset is quite big so we MUST restrict to the higher zoom level (ZARR do not have overviews)
maxzoom = r["maxzoom"]

m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=maxzoom
)

TileLayer(
    tiles=r["tiles"][0],
    opacity=1,
    attr="Noaa",
    min_native_zoom=maxzoom,
    max_native_zoom=maxzoom,
).add_to(m)

m

{'tilejson': '2.2.0', 'version': '1.0.0', 'scheme': 'xyz', 'tiles': ['https://t4z3zffpeh.execute-api.us-east-1.amazonaws.com/md/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?url=s3%3A%2F%2Fmur-sst%2Fzarr-v1&variable=analysed_sst&rescale=250%2C350&colormap_name=viridis'], 'minzoom': 0, 'maxzoom': 6, 'bounds': [-179.99500549324037, -89.99499786365084, 180.0050000000763, 89.99499786365084], 'center': [0.00499725341796875, 0.0, 0]}
